In [1]:
from aitemplates import create_chat_completion, Message, ChatSequence

message1 = Message('user', 'Can you tell me the price of a stock?')
sequence1 = ChatSequence([message1])
create_chat_completion(sequence1)

Total running cost: $0.000


"I'm sorry, but as an AI language model, I don't have real-time data or the ability to browse the internet. Therefore, I cannot provide you with the current price of a stock. However, you can easily find the price of a stock by using financial websites, stock market apps, or by contacting your broker."

In [2]:
from aitemplates import get_embedding

embedding = get_embedding('Can you tell me the price of a stock?')

print(f" [{embedding[0]},{embedding[1]}...{embedding[-1]}] {len(embedding)}")

Total running cost: $0.000
 [-0.008373594842851162,-0.008980377577245235...-0.004793579690158367] 1536


In [3]:
from aitemplates import async_create_chat_completion
from aitemplates import ChatConversation


message2 = Message('user', 'Can you tell me how much stock I can afford to buy with my budget?')
sequence2 = ChatSequence([message2])
chat=ChatConversation([sequence1, sequence2])
async_generate = await async_create_chat_completion(chat, keep_order=True)

100%|██████████| 2/2 [00:02<00:00,  1.24s/it]

Total running cost: $0.000


In [4]:
async_generate.display_conversation()

user: Can you tell me the price of a stock?

Response: As an AI language model, I do not have access to real-time stock market data. Please check a financial website or consult a financial advisor for the current price of a stock. 


user: Can you tell me how much stock I can afford to buy with my budget?

Response: As an AI language model, I do not have access to your budget information. Please provide me with your budget and the current stock price you are interested in, and I can help you calculate the number of shares you can afford to buy. 




In [5]:
from aitemplates import async_create_chat_completion


chat2 = ChatConversation([sequence1, sequence2])
async_generate = await async_create_chat_completion(chat, print_every=True)

 50%|█████     | 1/2 [00:01<00:01,  1.82s/it]

Total running cost: $0.000
As an AI language model, I do not have access to real-time stock market data. Please check a financial website or consult a financial advisor for the current price of a stock.



100%|██████████| 2/2 [00:02<00:00,  1.21s/it]

Total running cost: $0.000
As an AI language model, I do not have access to your budget information. Please provide me with your budget and the current stock price you are interested in, and I can help you calculate the number of shares you can afford to buy.



In [6]:
# first lets define our functions
stock_price = {
        "name": "get_company_stock_price",
        "description": "Use this function to get the price of a stock.",
        "parameters": {
            "type": "object",
            "properties": {
                "company": {
                    "type": "string",
                    "description": "The name of the company",
                },
            },
            "required": ["company"]
        },
    }

shares_calculation = {
        "name": "get_shares_calculation",
        "description": "Use this function to calculate how many shares you can buy with a certain budget, given a stock and a price of the stock",
        "parameters": {
            "type": "object",
            "properties": {
                "company": {
                    "type": "integer",
                    "description": "The name of the company",
                },
                "price": {
                    "type": "integer",
                    "description": "The price of the stock",
                },
                "budget": {
                    "type": "integer",
                    "description": "What your budget for stock purchases is",
                },
            },
            "required": ["company", "budget"]
        },
    }

In [7]:
from typing import Optional


# next, we need to define our functions so that they can be retrieved once a definition matches
def get_company_stock_price(company: str) -> float:
    """Get the stock price of a specified company."""

    print(company)
    # implement logic here to call the api 
    
    return 321

def get_shares_calculation(company: str, budget: int, price: Optional[int] = None) -> float:
    """Calculate how many shares I can buy for a given stock"""
    
    current_price = price
    
    if current_price is None:
        # Get the current stock price of the company
        current_price = get_company_stock_price(company)

    # Calculate the number of shares that can be bought with the budget
    return budget / current_price

In [8]:
from aitemplates import FunctionsAvailable, FunctionPair

# match the description to the function
function_pair1 = FunctionPair(stock_price, get_company_stock_price)

# next, let's add them to the FunctionsAvailable dataclass for easy access
functions_available = FunctionsAvailable([function_pair1])

sequence1.function_pairs.add_function_pairs(functions_available)

# if you pass in a function_call, it will call the function with assumptions
create_chat_completion(sequence1, function_call={"name": "get_company_stock_price"}, send_object=True)

Apple
Total running cost: $0.001


(<OpenAIObject chat.completion id=chatcmpl-7SE0RaJApzweMgQNzOjq25kuDXej1 at 0x29400fff5e0> JSON: {
   "id": "chatcmpl-7SE0RaJApzweMgQNzOjq25kuDXej1",
   "object": "chat.completion",
   "created": 1686961379,
   "model": "gpt-3.5-turbo-0613",
   "choices": [
     {
       "index": 0,
       "message": {
         "role": "assistant",
         "content": null,
         "function_call": {
           "name": "get_company_stock_price",
           "arguments": "{\n  \"company\": \"Apple\"\n}"
         }
       },
       "finish_reason": "stop"
     }
   ],
   "usage": {
     "prompt_tokens": 77,
     "completion_tokens": 9,
     "total_tokens": 86
   }
 },
 321)

In [9]:
from aitemplates import ResponseDict

# otherwise if the request matches one of the functions, it will ask you for clarification if it needs
response = create_chat_completion(sequence1, send_object=True)

# if function was called this is (ResponseDict, function_response), else it just returns a ResponseDict
response_tuple = ResponseDict.convert_to_response_dict(response)
response_tuple

Total running cost: $0.001


(ResponseDict(index=0, message=Message(role='assistant', content='Sure, I can help you with that. Please provide me with the name of the company whose stock price you would like to know.', function_call=None), finish_reason='stop', model='gpt-3.5-turbo-0613', created=1686961380),
 <OpenAIObject chat.completion id=chatcmpl-7SE0SGZeuDq15O508amYCraBvkKb8 at 0x29402e11b80> JSON: {
   "id": "chatcmpl-7SE0SGZeuDq15O508amYCraBvkKb8",
   "object": "chat.completion",
   "created": 1686961380,
   "model": "gpt-3.5-turbo-0613",
   "choices": [
     {
       "index": 0,
       "message": {
         "role": "assistant",
         "content": "Sure, I can help you with that. Please provide me with the name of the company whose stock price you would like to know."
       },
       "finish_reason": "stop"
     }
   ],
   "usage": {
     "prompt_tokens": 69,
     "completion_tokens": 28,
     "total_tokens": 97
   }
 })

In [10]:
# lets respond to the message above with the answer
response_dict = response_tuple[0]
llm_response = ChatSequence([response_dict.message])

chat_response = ChatSequence([Message('user', 'The Gamestop stock please')])

# you can see that we are passing the llm_response concatenated with our first and second queries
# previously we didn't pass in the functions globally, if you want certain functions to only apply to certain prompts add the function_pairs to those prompts
response2 = create_chat_completion(sequence1 + llm_response + chat_response, functions=functions_available, send_object=True)
response2 = ResponseDict.convert_to_response_dict(response2)
response2 

Gamestop
Total running cost: $0.001


(ResponseDict(index=0, message=Message(role='assistant', content=None, function_call=<OpenAIObject at 0x29402e5c130> JSON: {
   "name": "get_company_stock_price",
   "arguments": "{\n  \"company\": \"Gamestop\"\n}"
 }), finish_reason='function_call', model='gpt-3.5-turbo-0613', created=1686961381),
 321)

In [11]:
from aitemplates import ChatPair
# we can also chain these together using a handy ChatConversation class we defined earlier when working with async functions
# let's ask another question

# now lets chain a few of these together using the ChatConversation we defined
function_pair2 = FunctionPair(shares_calculation, get_shares_calculation)

functions_available.add_function_pairs(function_pair2)

existing_chat = ChatPair(sequence1, response_dict)

next_msg_response_pair = ChatPair(sequence2)

# this will create a list of tuples (ChatSequence, None)
chat = ChatConversation([existing_chat, next_msg_response_pair])

create_chat_completion(chat, functions=functions_available)

Total running cost: $0.001


'Sure, I can help you with that. Could you please provide me with the following information:\n\n1. The name of the company you are interested in.\n2. Your budget for stock purchases.'

In [12]:
# this is also now added to the chat conversation automatically
chat.display_conversation() 

user: Can you tell me the price of a stock?

Response: Sure, I can help you with that. Please provide me with the name of the company whose stock price you would like to know. 


user: Can you tell me how much stock I can afford to buy with my budget?

Response: Sure, I can help you with that. Could you please provide me with the following information:

1. The name of the company you are interested in.
2. Your budget for stock purchases. 




In [13]:
# finally, we respond with clarification so that it can use the second function we passed 
response_to_info_request = ChatSequence([Message('user', 'I told you, Gamestop is the stock... the budget it 42000')])

chat.add_pair(response_to_info_request)

response3 = create_chat_completion(chat, functions=functions_available)
response3

Gamestop
Total running cost: $0.001


130.84112149532712

In [14]:
chat.display_conversation()

user: Can you tell me the price of a stock?

Response: Sure, I can help you with that. Please provide me with the name of the company whose stock price you would like to know. 


user: Can you tell me how much stock I can afford to buy with my budget?

Response: Sure, I can help you with that. Could you please provide me with the following information:

1. The name of the company you are interested in.
2. Your budget for stock purchases. 


user: I told you, Gamestop is the stock... the budget it 42000

Response: 130.84112149532712 




In [18]:
from aitemplates import SingleApiManager

api_manager = SingleApiManager()
api_manager.total_cost

0.0008147